In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in the raw data.

# First Download dataset from https://www.kaggle.com/datasets/maxhorowitz/nflplaybyplay2009to2016?select=NFL+Play+by+Play+2009-2018+%28v5%29.csv
df = pd.read_csv('NFL Play by Play 2009-2018 (v5).csv', low_memory = False)
# A copy data frame to make changes to.
dfTrimmed = df.copy(deep=True)

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
dfTrimmed.drop([column for column in df.columns if column.endswith(('epa', 'prob', 'wpa', 'wp'))], axis=1, inplace=True)

In [5]:
dfTrimmed.drop([column for column in dfTrimmed.columns if dfTrimmed[column].count() < 100000], axis=1, inplace=True)

In [6]:
dfTrimmed.drop([column for column in dfTrimmed.columns if not (column=='game_id') and not (column=='away_wp') and column.endswith(('id', 'name', 'ep', '_wp', 'post'))], axis=1, inplace=True)

In [7]:
keep_cols = ['home_team', 'away_team', 'game_date', 'play_type', 'posteam', 'defteam', 'penalty_team']

In [8]:
# Drop columns with dtype = object
dfTrimmed.drop([column for column in dfTrimmed.columns if not (column in keep_cols) and dfTrimmed[column].dtype == object], axis=1, inplace=True)

In [9]:
# Remove columns that contain the strings 'timeout', 'out_of_bounds', or 'fair_catch'.
dfTrimmed.drop([column for column in dfTrimmed.columns if 'timeout' in column or 'out_of_bounds' in column or 'fair_catch' in column or '_remaining' in column or 'replay' in column or 'defensive' in column], axis=1, inplace=True)

In [10]:
to_remove=['return_yards', 'ydsnet', 'lateral_recovery', 'lateral_return', 'lateral_rush', 'lateral_reception', 'assist_tackle', 'field_goal_attempt', 'two_point_attempt', 'extra_point_attempt', 'return_touchdown', 'rush_touchdown', 'pass_touchdown', 'touchdown', 'own_kickoff_recovery_td', 'safety', 'fumble_not_forced', 'fumble_forced', 'kickoff_downed', 'punt_downed', 'score_differential', 'no_huddle', 'shotgun']
dfTrimmed.drop(to_remove, axis=1, inplace=True)

In [11]:
to_remove = ['kickoff_attempt', 'fumble_lost']
dfTrimmed.drop([column for column in dfTrimmed.columns if column in to_remove or column.startswith('qb_')], axis=1, inplace=True)

In [12]:
dfTrimmed.fillna(value=0, axis=0, inplace=True)

In [13]:
# Once again a new data frame is created to make changes on the data. This time the changes
# will be performed mostly on the rows. 
newdf = pd.DataFrame(columns=dfTrimmed.columns, dtype=np.float32)
# Create a column to store only unique game ids. We want one row per game.
newdf.game_id = dfTrimmed.game_id.unique()

In [14]:
# The attribute for each of the following 6 columns of data (for the first 
# instance of each unique game id) will be copied to the new data frame. 
# Home and away scores are set to the maximum value of each column respectively.
ids = newdf.game_id
homeTeam = [] # Current home team
awayTeam = [] # Current away team
posessionTeam = [] # Current team in possesion 
defensiveTeam = [] # Current defensive team
homeScore = [] # Final score of home team
awayScore = [] # Final score of away team

for i in ids:
    homeTeam.append(dfTrimmed[dfTrimmed.game_id == i].reset_index().home_team[0])
    awayTeam.append(dfTrimmed[dfTrimmed.game_id == i].reset_index().away_team[0])
    posessionTeam.append(dfTrimmed[dfTrimmed.game_id == i].reset_index().posteam[0])
    defensiveTeam.append(dfTrimmed[dfTrimmed.game_id == i].reset_index().defteam[0])
    homeScore.append(max(dfTrimmed[dfTrimmed.game_id == i].reset_index().total_home_score))
    awayScore.append(max(dfTrimmed[dfTrimmed.game_id == i].reset_index().total_away_score))
    
newdf.home_team = homeTeam
newdf.away_team = awayTeam
newdf.posteam = posessionTeam
newdf.defteam = defensiveTeam
newdf.total_home_score = homeScore
newdf.total_away_score = awayScore


In [15]:
# The following attributes require a summation over the entire column for each unique game. 
# Each of the corresponding columns are summed over and a distinction is made between each of the
# two teams in each game. The same process takes place in the next cell.
ids = newdf.game_id
home4th = [] # Total 4th down conversions for home team
away4th = [] # Total 4th down conversions for away team 
homeInt = [] # Total interceptions thrown by home team
awayInt = [] # Total interceptions thrown by away team
homeOKO = [] # Own kickoff recoveries for home team
awayOKO = [] # Own kickoff recoveries for away team
homeS = [] # Defensive sacks for the home team (Defensive stat)
awayS = [] # Defensive sacks for the away team (Defensive stat)
homePyrds = [] # Penalty yards accumulated by the home team
awayPyrds = [] # Penalty yeards accumulated by the away team
homeBlocked = [] # Defensive blocked punts for the home team (Defensive stat)
awayBlocked = []  # Defensive blocked punts for the away team (Defensive stat)
for i in ids:
    teams = (dfTrimmed[dfTrimmed.game_id == i].reset_index().home_team[0], dfTrimmed[dfTrimmed.game_id == i].reset_index().away_team[0])


    count = 0
    for team in teams:
        if count == 0:
            
            home4th.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().fourth_down_converted))
            homeInt.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().interception))
            homeOKO.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().own_kickoff_recovery))
            homeS.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().sack))
            
            homeBlocked.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().punt_blocked))

        else:
            away4th.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().fourth_down_converted))
            awayInt.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().interception))
            awayOKO.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().own_kickoff_recovery))
            awayS.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().sack))
           
            awayBlocked.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().punt_blocked))
        count+=1
                                                    
newdf.fourth_down_converted = home4th
newdf['fourth_down_converted_away'] = away4th
newdf.interception = homeInt
newdf['interception_away'] = awayInt
newdf.own_kickoff_recovery = homeOKO
newdf['own_kickoff_recovery_away'] = awayOKO
newdf.sack = homeS
newdf['sack_away'] = awayS
newdf.punt_blocked = homeBlocked
newdf['punt_blocked_away'] = awayBlocked  


In [16]:
homePI20 = [] # Punts inside the 20 for the home team
awayPI20 = [] # Punts inside the 20 for the away team
homePIEZ = [] # Punts in endzone for the home team
awayPIEZ = [] # Punts in endzone for the away team
homeTotalYards = [] # Home team total yards
awayTotalYards = [] # Away team total yards
homeCompPercentage = [] # Home team QB pass completion percentage
awayCompPercentage = [] # Away team QB pass completion percentage
homeRushAttempts = [] # Number of rushing attempts for the home team
awayRushAttempts = [] # Number of rushing attempts for the away team
homeQBhit = [] # QB hits for the home team (Defensive stat)
awayQBhit = [] # QB hits for the away team (Defensive stat)
home3rd = [] # Total 3rd down conversions for home team
away3rd = [] # Total 3rd down conversions for away team
ids = newdf.game_id
for i in ids:
    teams = (dfTrimmed[dfTrimmed.game_id == i].reset_index().home_team[0], dfTrimmed[dfTrimmed.game_id == i].reset_index().away_team[0])

    count = 0
    for team in teams:
        if count == 0:
            homePI20.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().punt_inside_twenty))
            homePIEZ.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().punt_in_endzone))
            homeTotalYards.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().yards_gained))
            homeCompPercentage.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().complete_pass) /
                                     (sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().incomplete_pass) +
                                      sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().complete_pass)))
            homeRushAttempts.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().rush_attempt))
            
            home3rd.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().third_down_converted))
        else:
            awayPI20.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().punt_inside_twenty))
            awayPIEZ.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().punt_in_endzone))
            awayTotalYards.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().yards_gained))
            awayCompPercentage.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().complete_pass) /
                                     (sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().incomplete_pass) +
                                      sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().complete_pass)))
            awayRushAttempts.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().rush_attempt))
            
            away3rd.append(sum(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().third_down_converted))
        count+=1
                                  
newdf.punt_inside_twenty = homePI20
newdf.punt_in_endzone = homePIEZ
newdf['punt_inside_twenty_away'] = awayPI20
newdf['punt_in_endzone_away'] = awayPIEZ
newdf.yards_gained = homeTotalYards
newdf['yards_gained_away'] = awayTotalYards
newdf['Pass_comp_percentage'] = homeCompPercentage
newdf['Pass_comp_percentage_away'] = awayCompPercentage
newdf.rush_attempt = homeRushAttempts
newdf['rush_attempt_away'] = awayRushAttempts

newdf.third_down_converted = home3rd
newdf['third_down_converted_away'] = away3rd

In [17]:
to_remove = ['yardline_100', 'quarter_end', 'down', 'goal_to_go', 'ydstogo', 'play_type', 'posteam_score', 'defteam_score', 'incomplete_pass', 'penalty', 'pass_attempt', 'complete_pass', 'qtr', 'air_yards', 'sp', 'first_down_penalty', 'yards_after_catch']
newdf.drop(to_remove, axis=1, inplace=True)


In [18]:
homeDrives = [] # Number of drives for the home team
awayDrives = [] # Number of drives for the away team
homeTackledForLoss = [] # Home team tackled for loss
awayTackledForLoss = [] # Away team tackled for loss
homeFirstDownRush = [] # Home team first down rush
awayFirstDownRush = [] # Away team first down rush
homeFirstDownPass = [] # Home team first down pass
awayFirstDownPass = [] # Away team first down pass
homeThirdDownFailed = [] # Home team failed third down conversions
awayThirdDownFailed = [] # Away team failed third down conversions
homeFourthDOwnFailed = [] # Home team failed fourth down conversions
awayFourthDOwnFailed = [] # Away team failed fourth down conversions
homeKickoffInside20 = [] # Home team kickoff inside 20
awayKickoffInside20 = [] # Away team kickoff inside 20
homeKickoffInEndzone = [] # Home team kickoff in endzone
awayKickoffInEndzone = [] # Away team kickoff in endzone
homeSoloTackles = [] # Home team solo tackles
awaySoloTackles = [] # Away team solo tackles
homeTacklesForLoss = [] # Home team tackles for loss
awayTacklesForLoss = [] # Away team tackles for loss
homePuntAttempts = [] # Home team punt attempts
awayPuntAttempts = [] # Away team punt attempts
homeFumbles = [] # Home team fumbles
awayFumbles = [] # Away team fumbles


ids = newdf.game_id
for i in ids:
    teams = (dfTrimmed[dfTrimmed.game_id == i].reset_index().home_team[0], dfTrimmed[dfTrimmed.game_id == i].reset_index().away_team[0])

    count = 0
    for team in teams:
        if count == 0:
            homeDrives.append(max(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().drive) // 2)
            homeTackledForLoss.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().tackled_for_loss))
            homeFirstDownRush.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().first_down_rush))
            homeFirstDownPass.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().first_down_pass))
            homeThirdDownFailed.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().third_down_failed))
            homeFourthDOwnFailed.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().fourth_down_failed))
            homeKickoffInside20.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().kickoff_inside_twenty))
            homeKickoffInEndzone.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().kickoff_in_endzone))
            homeSoloTackles.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().solo_tackle))
            homeTacklesForLoss.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().tackled_for_loss))
            homePuntAttempts.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().punt_attempt))
            homeFumbles.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().fumble))
            
        else:
            awayDrives.append(max(dfTrimmed[(dfTrimmed.game_id == i) & 
                                         (dfTrimmed.posteam == team)].reset_index().drive) // 2)
            awayTackledForLoss.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().tackled_for_loss))
            awayFirstDownRush.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().first_down_rush))
            awayFirstDownPass.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().first_down_pass))
            awayThirdDownFailed.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().third_down_failed))
            awayFourthDOwnFailed.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().fourth_down_failed))
            awayKickoffInside20.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().kickoff_inside_twenty))
            awayKickoffInEndzone.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().kickoff_in_endzone))
            awaySoloTackles.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().solo_tackle))
            awayTacklesForLoss.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().tackled_for_loss))
            awayPuntAttempts.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().punt_attempt))
            awayFumbles.append(sum(dfTrimmed[(dfTrimmed.game_id == i) &
                                            (dfTrimmed.posteam == team)].reset_index().fumble))

        count += 1

newdf.drive = homeDrives
newdf.first_down_rush = homeFirstDownRush
newdf.first_down_pass = homeFirstDownPass
newdf.third_down_failed = homeThirdDownFailed
newdf.fourth_down_failed = homeFourthDOwnFailed
newdf.kickoff_inside_twenty = homeKickoffInside20
newdf.kickoff_in_endzone = homeKickoffInEndzone
newdf.solo_tackle = homeSoloTackles
newdf.tackled_for_loss = homeTacklesForLoss
newdf.punt_attempt = homePuntAttempts
newdf.fumble = homeFumbles

newdf['away_drive'] = awayDrives
newdf['away_first_down_rush'] = awayFirstDownRush
newdf['away_first_down_pass'] = awayFirstDownPass
newdf['away_third_down_failed'] = awayThirdDownFailed
newdf['away_fourth_down_failed'] = awayFourthDOwnFailed
newdf['away_kickoff_inside_twenty'] = awayKickoffInside20
newdf['away_kickoff_in_endzone'] = awayKickoffInEndzone
newdf['away_solo_tackle'] = awaySoloTackles
newdf['away_tackled_for_loss'] = awayTacklesForLoss
newdf['away_punt_attempt'] = awayPuntAttempts
newdf['away_fumble'] = awayFumbles


In [19]:
dates=[]
ids = newdf.game_id
for i in ids:
    dates.append(dfTrimmed[dfTrimmed.game_id == i].reset_index().game_date[0])

newdf.game_date = dates


In [20]:
newdf.head(3)

,game_id,home_team,away_team,posteam,defteam,game_date,drive,yards_gained,total_home_score,total_away_score,punt_blocked,first_down_rush,first_down_pass,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,interception,punt_inside_twenty,punt_in_endzone,kickoff_inside_twenty,kickoff_in_endzone,solo_tackle,tackled_for_loss,own_kickoff_recovery,rush_attempt,sack,punt_attempt,fumble,fourth_down_converted_away,interception_away,own_kickoff_recovery_away,sack_away,punt_blocked_away,punt_inside_twenty_away,punt_in_endzone_away,yards_gained_away,Pass_comp_percentage,Pass_comp_percentage_away,rush_attempt_away,third_down_converted_away,away_drive,away_first_down_rush,away_first_down_pass,away_third_down_failed,away_fourth_down_failed,away_kickoff_inside_twenty,away_kickoff_in_endzone,away_solo_tackle,away_tackled_for_loss,away_punt_attempt,away_fumble
0,2009091000,PIT,TEN,PIT,TEN,2009-09-10,12,357.0,13,10,0.0,1.0,18.0,4.0,10.0,0.0,0.0,2.0,4.0,0.0,0.0,2.0,41.0,3.0,0.0,23.0,4.0,7.0,2.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,320.0,0.804878,0.647059,25.0,4.0,12,2.0,14.0,9.0,0.0,0.0,1.0,35.0,5.0,5.0,1.0
1,2009091304,CLE,MIN,CLE,MIN,2009-09-13,12,268.0,19,34,0.0,5.0,10.0,3.0,9.0,0.0,1.0,1.0,2.0,0.0,2.0,3.0,44.0,2.0,0.0,20.0,5.0,5.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,310.0,0.617647,0.636364,37.0,6.0,12,14.0,4.0,8.0,0.0,0.0,1.0,44.0,3.0,6.0,0.0
2,2009091307,NO,DET,NO,DET,2009-09-13,13,515.0,45,26,0.0,10.0,16.0,9.0,4.0,0.0,0.0,1.0,0.0,0.0,3.0,2.0,54.0,3.0,0.0,35.0,0.0,2.0,1.0,2.0,3.0,0.0,1.0,0.0,2.0,0.0,231.0,0.787879,0.470588,20.0,3.0,12,5.0,7.0,11.0,0.0,2.0,3.0,38.0,2.0,5.0,2.0


In [21]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2526 entries, 0 to 2525
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   game_id                     2526 non-null   int64  
 1   home_team                   2526 non-null   object 
 2   away_team                   2526 non-null   object 
 3   posteam                     2526 non-null   object 
 4   defteam                     2526 non-null   object 
 5   game_date                   2526 non-null   object 
 6   drive                       2526 non-null   int64  
 7   yards_gained                2526 non-null   float64
 8   total_home_score            2526 non-null   int64  
 9   total_away_score            2526 non-null   int64  
 10  punt_blocked                2526 non-null   float64
 11  first_down_rush             2526 non-null   float64
 12  first_down_pass             2526 non-null   float64
 13  third_down_converted        2526 

In [22]:
newdf.to_csv('newData.csv')